In [1]:
#setup

import io
import os
import re
import shutil
import string
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 26s 0us/step


In [5]:
dataset_dir = os.path.join(os.path.dirname(dataset),"aclImdb")

In [6]:
train_dir = os.path.join(dataset_dir, "train")

In [7]:
test_dir = os.path.join(dataset_dir, "test")

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32 
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size
)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


In [14]:
#let's look at the data
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]} \n')
        label = label_batch.numpy()[i]
        print(f'Review: {class_names[label]} \n')

Review: b"I bought this thinking I was gonna see Olivier Gruner kick some butt for 90 minutes but no the reasonably impressive cast including Michael Madson, Michael Ironside and Martin Kove are all wasted in mere cameos as they make way for the introduction of new action star Dan Anderson. The film also known in another name as Last line of defence 2 an apparant sequal to another film Gruner was in as the lead as a different character. What is the point? the first which I have not seen is apparantly about an alien or something, this is completley different featuring a story about a disgraced army guy (can't remeber who he worked for) who needs to rasie 5 million for a special new cancer treatment that will save his son and other children. Michael Ironside plays a wealthy business man who is of course a dirty scoundral who is swimming in dirty ill-gotten money. He laughs at Dan Andersons character and refuses to give it. Dan decides to steal it from him with the help of some ex colleag